In [34]:
import panel as pn
pn.extension('tabulator', sizing_mode='stretch_width', notifications=True)
from panel.io.notifications import NotificationArea
NotificationArea.position = 'bottom-right'

import pyvo as vo
import datetime as dt
from astropy.time import Time
import param
from pypika import Table, Criterion, EmptyCriterion, Order
import pandas as pd
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astropy import units as u
import warnings
from astroquery.exceptions import TableParseError
from astropy.io import fits
from matplotlib.figure import Figure
from matplotlib import cm
import numpy as np
from pathlib import Path
from astropy.coordinates import Angle

#NAXIS1 IS X IS RA
#NAXIS2 IS Y IS DEC

NotificationArea(sizing_mode='stretch_width')

In [2]:
url = "https://vo.astro.rug.nl/tap"
service = vo.dal.TAPService(url)

In [3]:
from pypika.queries import Query, QueryBuilder
from pypika.utils import builder, QueryException
from typing import Any, Optional
from enum import Enum

class Dialects(Enum):
    ADQL = 'adql'

class ADQLQuery(Query):
    @classmethod
    def _builder(cls, **kwargs: Any) -> "QueryBuilder":
        return ADQLQueryBuilder(**kwargs)

class ADQLQueryBuilder(QueryBuilder):
    QUOTE_CHAR = None
    ALIAS_QUOTE_CHAR = '"'
    QUERY_ALIAS_QUOTE_CHAR = ''
    QUERY_CLS = ADQLQuery

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(dialect=Dialects.ADQL, **kwargs)
        self._top = None
        self._contains_point = None
        self._contains_box = None

    @builder
    def top(self, value):
        try:
            self._top = int(value)
        except ValueError:
            raise QueryException('TOP value must be an integer')

    def get_sql(self, *args, **kwargs):
        return super(ADQLQueryBuilder, self).get_sql(*args, groupby_alias=False, **kwargs)

    def _top_sql(self):
        if self._top:
            return 'TOP {} '.format(self._top)
        else:
            return ''

    @builder
    def contains_point(self, ra, dec, from_table, cone_radius):
        self._contains_point = [ra, dec, from_table, cone_radius]

    def _contains_point_sql(self):
        if self._contains_point:
            return "CONTAINS(POINT('ICRS', {}, {}), CIRCLE('ICRS', {}.ra, {}.dec, {}))=1 AND ".format(
                self._contains_point[0], self._contains_point[1], self._contains_point[2], self._contains_point[2], self._contains_point[3])
        else:
            return ''

    @builder
    def contains_box(self, ra, dec, from_table, width, height):
        self._contains_box = [ra, dec, from_table, width, height]

    def _contains_box_sql(self):
        if self._contains_box:
            return "CONTAINS(POINT('ICRS', {}, {}), BOX('ICRS', {}.ra, {}.dec, {}, {}))=1 AND ".format(
                self._contains_box[0], self._contains_box[1], self._contains_box[2], self._contains_box[2], self._contains_box[3], self._contains_box[4])
        else:
            return ''

    def _select_sql(self, **kwargs: Any) -> str:
        return "SELECT {distinct}{top}{select}".format(
            top=self._top_sql(),
            distinct="DISTINCT " if self._distinct else "",
            select=",".join(term.get_sql(with_alias=True, subquery=True, **kwargs) for term in self._selects)
        )

    @builder
    def _where_sql(self, quote_char: Optional[str] = None, **kwargs: Any) -> str:
        return " WHERE {contains_point}{contains_box}{where}".format(
            contains_point=self._contains_point_sql(),
            contains_box = self._contains_box_sql(),
            where=self._wheres.get_sql(quote_char=quote_char, subquery=True, **kwargs)
        )

In [74]:
options = {'Filename': 'filename',
           'Observation date': 'kw_DATE_OBS',
           'Image type': 'kw_IMAGETYP',
           'Right ascension': 'ra',
           'Declination': 'dec',
           'Object': 'kw_OBJECT',
           'Filter': 'kw_FILTER'}
options_reverse = {'filename': 'Filename',
                   'kw_DATE_OBS': 'Observation date',
                   'kw_IMAGETYP': 'Image type',
                   'ra': 'Right ascension',
                   'dec': 'Declination',
                   'kw_OBJECT': 'Object',
                   'kw_FILTER': 'Filter'}
class CompileQuery(param.Parameterized):
    begin_date = dt.datetime(2008, 5, 1, 12)
    end_date = dt.datetime(2020, 4, 23, 12)
    dates = param.DateRange((begin_date, end_date), precedence=1)
    dates_order = param.Selector({'Descending': Order.desc, 'Ascending': Order.asc}, precedence=1)
    reset_date_button = param.Action(lambda x: x.param.trigger('reset_date_button'), label='Reset date', precedence=1)
    select_columns = param.ListSelector(default=['Filename',
                                                 'Image type',
                                                 'Filter',
                                                 'Observation date',
                                                 'Right ascension',
                                                 'Declination',
                                                 'Object'],
                                        objects=options.keys(),
                                        precedence=1)
    ra_dec_not_null = param.Boolean(default=True, precedence=1)
    object_or_coordinates = param.Selector(['Object', 'Coordinates'], precedence=1)
    select_object = param.String(default='', precedence=1)
    search_coordinates = param.String(default='' , precedence=-1)
    box_or_cone = param.Selector(['Box', 'Cone'], precedence=1)
    cone_radius = param.Number(default=20, bounds=(5, 40), precedence=-1)
    box_width = param.Number(1700, precedence=1)
    box_height = param.Number(1100, precedence=1)
    nr_entries = param.Number(100, precedence=1)

    @param.depends('box_or_cone', watch=True)
    def box_cone_search(self):
        if self.box_or_cone == 'Box':
            self.param.cone_radius.precedence = -1
            self.param.box_width.precedence = 1
            self.param.box_height.precedence = 1
        else:
            self.param.cone_radius.precedence = 1
            self.param.box_width.precedence = -1
            self.param.box_height.precedence = -1

    @param.depends('object_or_coordinates', watch=True)
    def object_coordinates_search(self):
        if self.object_or_coordinates == 'Object':
            self.param.select_object.precedence = 1
            self.param.search_coordinates.precedence = -1
        else:
            self.select_object = ''
            self.param.select_object.precedence = -1
            self.param.search_coordinates.precedence = 1

    @param.depends('reset_date_button', watch=True)
    def reset_date(self):
        self.dates = (dt.datetime(2008, 5, 1, 12), dt.datetime(2022, 5, 1, 12))

    def table_string(self):
        table_name = 'observations.raw'
        return table_name

    def from_query(self):
        raw_observations = Table(self.table_string())
        return raw_observations

    @param.depends('select_columns')
    def select_query(self):
        select_options = ''
        for i in self.select_columns:
            select_options += str(self.from_query()).strip('"') + '.' + options[i] + ','
        return select_options.rstrip(',')

    @param.depends('select_object')
    def find_object(self):
        if self.select_object == '':
            return True
        else:
            object_name = self.select_object
            with warnings.catch_warnings():
                warnings.filterwarnings('error')
                try:
                    simbad_object = Simbad.query_object(object_name)
                    coordinates = SkyCoord(simbad_object['RA'], simbad_object['DEC'], unit=(u.hourangle, u.deg))
                    object_ra, object_dec = coordinates.ra.deg[0], coordinates.dec.deg[0]
                    return object_ra, object_dec
                except (Warning, TableParseError):
                    return False

    @param.depends('search_coordinates')
    def coordinates_to_degrees(self):
        if self.search_coordinates == '':
            return
        right_ascension, declination = self.search_coordinates.split(',')
        if len(right_ascension) > 1:
            if float(right_ascension.split(' ')[0]) >= 24:
                right_ascension = Angle(right_ascension, unit=u.degree)
                right_ascension.wrap_at(360 * u.degree, inplace=True)
            else:
                right_ascension = Angle(right_ascension, unit=u.hourangle)
                right_ascension.wrap_at(24 * u.hourangle, inplace=True)
        else:
            if float(right_ascension) >= 24:
                right_ascension = Angle(right_ascension, unit=u.degree)
                right_ascension.wrap_at(360 * u.degree, inplace=True)
            else:
                right_ascension = Angle(right_ascension, unit=u.hourangle)
                right_ascension.wrap_at(24 * u.hourangle, inplace=True)
        declination = Angle(declination, unit=u.deg)
        right_ascension = right_ascension.deg
        declination = declination.deg
        if declination < -90:
            while declination < -90:
                declination += 90
        elif declination > 90:
            while declination > 90:
                declination -= 90
        return right_ascension, declination

    @param.depends('ra_dec_not_null')
    def not_null_check(self):
        if self.ra_dec_not_null:
            return True

    @param.depends('dates')
    def date_query(self):
        start_date = Time(self.dates[0]).jd
        end_date = Time(self.dates[1]).jd
        return start_date, end_date

In [75]:
# test = CompileQuery()
# test.coordinates_to_degrees()

In [118]:
class FetchData(CompileQuery):
    search_object_button = param.Action(lambda x: x.param.trigger('search_object_button'), label='Search', precedence=5)
    reset_search_button = param.Action(lambda x: x.param.trigger('reset_search_button'), label='Reset search', precedence=5)

    @param.depends('reset_search_button', watch=True)
    def reset_search(self):
        self.dates = (self.begin_date, self.end_date)
        self.ra_dec_not_null = True
        self.select_object = ''
        self.cone_radius = 20
        self.param.trigger('search_object_button')

    @param.depends('search_object_button')
    def data(self):
        if not self.find_object():
            return
        observation_table = self.from_query()
        dates = self.date_query()
        query = ADQLQuery.from_(
            observation_table
        ).select(
            self.select_query()
        ).where(
            Criterion.all([
                observation_table.ra.isnotnull() if self.not_null_check() else EmptyCriterion(),
                observation_table.dec.isnotnull() if self.not_null_check() else EmptyCriterion(),
                observation_table.obs_jd[dates[0]:dates[1]]
            ])
        ).orderby(observation_table.obs_jd, order=self.dates_order).top(self.nr_entries)
        if self.select_object != '':
            right_ascension, declination = self.find_object()
            if self.box_or_cone == 'Cone':
                query = query.contains_point(right_ascension, declination, self.table_string(), self.cone_radius/60)
            elif self.box_or_cone == 'Box':
                query = query.contains_box(self.find_object()[0], self.find_object()[1], self.table_string(), self.box_width/3600, self.box_height/3600)
        elif self.select_object == '' and self.search_coordinates != '':
            right_ascension, declination = self.coordinates_to_degrees()
            if self.box_or_cone == 'Cone':
                query = query.contains_point(right_ascension, declination, self.table_string(), self.cone_radius/60)
            elif self.box_or_cone == 'Box':
                query = query.contains_box(right_ascension, declination, self.table_string(), self.box_width/3600, self.box_height/3600)
        result = service.search(query.get_sql())
        data_pandas = result.to_table().to_pandas()
        data_pandas['ra'] = data_pandas['ra'].apply(self.ra_to_hms)
        data_pandas['dec'] = data_pandas['dec'].apply(self.dec_to_dms)
        return data_pandas

    def ra_to_hms(self, nr_angle):
        angle = Angle(nr_angle, u.degree)
        return angle.to_string(unit=u.hour, sep=('h:', 'm:', 's'), precision=3)

    def dec_to_dms(self, nr_angle):
        angle = Angle(nr_angle, u.degree)
        return angle.to_string(unit=u.degree, sep=('d:', 'm:', 's'), precision=3)

    @param.depends('search_object_button')
    def display_object_error(self):
        if not self.find_object():
            message = "SIMBAD could not find the object " + str(self.select_object) + " that you searched for."
            pn.state.notifications.error(message, duration=0)

    @param.depends('search_object_button', 'data')
    def display_data_error(self):
        if self.find_object() and self.data().values.size == 0:
            message = "No entries could be found with the options you specified."
            pn.state.notifications.error(message, duration=0)

In [322]:
class CreateTable(FetchData):
    table = param.DataFrame(columns=CompileQuery().select_columns, precedence=-1)
    if FetchData().data() is None:
        table.default = pd.DataFrame(columns=CompileQuery().select_columns)
    else:
        table.default = FetchData().data()
        table.columns = options_reverse

    def __init__(self, **params):
        super().__init__(**params)
        self.table_widget = pn.widgets.Tabulator(self.table,
                                                 disabled=True,
                                                 pagination='local',
                                                 show_index=False,
                                                 layout='fit_columns',
                                                 selectable='checkbox-single',
                                                 text_align='left',
                                                 min_height=500,
                                                 titles=options_reverse)

        self.selected_table_widget = pn.widgets.Tabulator(disabled=True,
                                                          pagination='local',
                                                          show_index=False,
                                                          layout='fit_data',
                                                          selectable='checkbox-single',
                                                          text_align='left',
                                                          min_height=500,
                                                          titles=options_reverse)

    @param.depends('data')
    def full_table(self):
        if self.data() is None:
            self.table_widget.value = pd.DataFrame(columns=self.select_columns)
        else:
            self.table_widget.value = self.data()
            self.table_widget.titles = options_reverse
        return self.table_widget

    @param.depends('table_widget.selected_dataframe')
    def selected_table(self):
        selection = self.table_widget.selected_dataframe
        if selection.values.size == 0:
            self.selected_table_widget.value = pd.DataFrame(columns=['filename', 'kw_FILTER', 'kw_OBJECT'])
        else:
            selection = selection[['filename', 'kw_FILTER', 'kw_OBJECT']]
            selection['filename'] = selection['filename'].apply(self.give_file_name)
            self.selected_table_widget.value = selection
        return self.selected_table_widget

    def give_file_name(self, filepath):
        return Path(filepath).name

In [ ]:
class CreatePlots(CreateTable):

    # @param.depends('selected_table_widget.selected_dataframe')
    # def create_plot(self):
    #     home_dir = '../Data/Raw/'
    #     filepath = home_dir + self.selected_table_widget.value['filename'].iloc[0]
    #     fits_file = fits.open(filepath)
    #     fits_data = fits_file[0].data
    #     vmin = np.percentile(fits_data, 5)
    #     vmax = np.percentile(fits_data, 95)
    #     img = self.ax.imshow(fits_data, interpolation='none', origin='lower', cmap=cm.gray, vmin=vmin, vmax=vmax)
    #     return pn.pane.Matplotlib(self.fig, tight=True, sizing_mode='scale_both')

    fits_file = fits.open('../Data/Raw/200422_Li_.00000040.NGC_5907.FIT')
    fits_data = fits_file[0].data
    headers = fits_file[0].header
    vmin = np.percentile(fits_data, 5)
    vmax = np.percentile(fits_data, 95)

    def create_plot(self):
        fig0 = Figure(figsize=(8, 6))
        ax0 = fig0.subplots()
        img = ax0.imshow(self.fits_data, interpolation='none', origin='lower', cmap=cm.gray, vmin=self.vmin, vmax=self.vmax)
        fig0.colorbar(img)
        return pn.pane.Matplotlib(fig0, tight=True, sizing_mode='scale_both')

In [ ]:
# test = CreatePlots()
# for key in test.headers:
#     print(key, test.headers[key])

In [ ]:
class CreateView(CreatePlots):
    def panel(self):
        widgets = {
            'dates': pn.widgets.DatetimeRangePicker,
            'dates_order': pn.widgets.Select,
            'reset_date_button': {'widget_type': pn.widgets.Button, 'button_type': 'warning'},
            'select_columns': {'widget_type': pn.widgets.CrossSelector, 'definition_order': False},
            'ra_dec_not_null': pn.widgets.Checkbox,
            'object_or_coordinates': {'widget_type': pn.widgets.RadioBoxGroup, 'inline': True},
            'select_object': pn.widgets.TextInput,
            'search_coordinates': {'widget_type': pn.widgets.TextInput, 'name': 'Coordinates (hh mm ss.ms, dd mm ss.ms)'},
            'box_or_cone': {'widget_type': pn.widgets.RadioBoxGroup, 'inline': True},
            'cone_radius': {'widget_type': pn.widgets.FloatSlider, 'name': 'Cone radius (arc minutes)' , 'step': 0.5},
            'box_width': {'widget_type': pn.widgets.FloatInput, 'name': 'Box width (arc seconds)'},
            'box_height': {'widget_type': pn.widgets.FloatInput, 'name': 'Box height (arc seconds)'},
            'nr_entries': {'widget_type': pn.widgets.FloatInput, 'name': '# of entries'},
            'search_object_button': {'widget_type': pn.widgets.Button, 'button_type': 'primary'},
            'reset_search_button': {'widget_type': pn.widgets.Button, 'button_type': 'warning'}
        }
        settings = pn.Row(
            pn.Param(self, widgets=widgets, width=385, sizing_mode="fixed", name="Settings")
        )

        # settings = pn.Column(
        #     pn.widgets.DatetimeRangePicker.from_param(self.param.dates),
        #     pn.widgets.Select.from_param(self.param.dates_order),
        #     pn.widgets.Button.from_param(self.param.reset_date_button, button_type='warning'),
        #     pn.widgets.CrossSelector.from_param(self.param.select_columns, definition_order=False),
        #     pn.widgets.Checkbox.from_param(self.param.ra_dec_not_null),
        #     pn.widgets.TextInput.from_param(self.param.select_object),
        #     pn.widgets.RadioBoxGroup.from_param(self.param.box_or_cone, inline=True),
        #     pn.widgets.FloatSlider.from_param(self.param.cone_radius, name='Cone radius (arc minutes)', step=0.5),
        #     pn.Row(
        #         pn.widgets.FloatInput.from_param(self.param.box_width, name='Box width (arc seconds)'),
        #         pn.widgets.FloatInput.from_param(self.param.box_height, name='Box height (arc seconds)'),
        #     ),
        #     pn.widgets.Button.from_param(self.param.search_object_button, button_type='primary'),
        #     pn.widgets.Button.from_param(self.param.reset_search_button, button_type='warning')
        # )
        bootstrap.sidebar.append(settings)
        tabs = pn.Tabs(
            ('Data Table', pn.Column(self.full_table, self.display_object_error, self.display_data_error)),
            ('Data Plotting', pn.Row(pn.Card(self.selected_table), pn.Card(self.create_plot)))
        )
        bootstrap.main.append(tabs)
        return bootstrap

In [246]:
bootstrap = pn.template.BootstrapTemplate(title='Bootstrap Template', sidebar_width=400)

test = CreateView()
test.panel().servable();
# print(test.selected_table().value)

In [192]:
# query = """
# SELECT TOP 10 id, ra, dec, d_PLATE_SCALE, kw_NAXIS1, kw_NAXIS2
# FROM observations.raw
# WHERE ra IS NOT NULL
# AND dec IS NOT NULL
# """
# result = service.search(query)
# table = result.to_table().to_pandas()
#
# def ra_to_hms(nr_angle):
#     angle = Angle(nr_angle, u.degree)
#     return angle.to_string(unit=u.hour)
#
# def dec_to_dms(nr_angle):
#     angle = Angle(nr_angle, u.degree)
#     return angle.to_string(unit=u.degree, sep=('d', 'm', 's'))
#
# table['ra'] = table['ra'].apply(ra_to_hms)
# table['dec'] = table['dec'].apply(dec_to_dms)
# table

,id,ra,dec,d_PLATE_SCALE,kw_NAXIS1,kw_NAXIS2
0,378,20h03m11.06118288s,58d57m57.91395276s,1.13256,1536,1024
1,379,20h03m11.05822824s,58d57m57.93165504s,1.13229,1536,1024
2,380,20h03m11.06467872s,58d57m57.92580324s,1.13261,1536,1024
3,381,20h03m11.05478688s,58d57m57.9426084s,1.13231,1536,1024
4,382,20h03m11.06029512s,58d57m57.91287672s,1.13234,1536,1024
5,383,20h03m11.05380696s,58d57m57.90242952s,1.13269,1536,1024
6,384,20h03m11.0636928s,58d57m57.94178364s,1.13255,1536,1024
7,385,20h02m12.79970016s,58d56m25.3914414s,1.13249,1536,1024
8,386,20h03m11.05394664s,58d57m57.88801512s,1.13233,1536,1024
9,387,20h03m11.05935648s,58d57m57.91380336s,1.13255,1536,1024


In [194]:
# print(dec_to_dms(58.966083))

58d57m57.8988s


In [15]:
# class Precedence(param.Parameterized):
#
#     # Parameters of the dummy app.
#     x = param.Number(precedence=-1)
#     y = param.Boolean(precedence=3)
#     z = param.String(precedence=2)
#
#     # Parameters of the control app.
#     x_precedence = param.Number(default=x.precedence, bounds=(-10, 10), step=1)
#     y_precedence = param.Number(default=y.precedence, bounds=(-10, 10), step=1)
#     z_precedence = param.Number(default=z.precedence, bounds=(-10, 10), step=1)
#     dummy_app_display_threshold = param.Number(default=1, bounds=(-10, 10), step=1)
#
#     def __init__(self):
#         super().__init__()
#         # Building the dummy app as a Param pane in here so that its ``display_threshold``
#         # parameter can be accessed and linked via @param.depends(...).
#         self.dummy_app = pn.Param(
#             self.param,
#             parameters=["x", "y", "z"],
#             widgets={
#                 "x": {"background": "#fac400"},
#                 "y": {"background": "#07d900"},
#                 "z": {"background": "#00c0d9"},
#             },
#             show_name=False
#         )
#
#     # Linking the two apps here.
#     @param.depends("dummy_app_display_threshold", "x_precedence", "y_precedence", "z_precedence", watch=True)
#     def update_precedences_and_threshold(self):
#         self.param.x.precedence = self.x_precedence
#         self.param.y.precedence = self.y_precedence
#         self.param.z.precedence = self.z_precedence
#         self.dummy_app.display_threshold = self.dummy_app_display_threshold
#
# precedence_model = Precedence()
#
# control_app = pn.Param(
#     precedence_model.param,
#     parameters=["x_precedence", "y_precedence", "z_precedence", "dummy_app_display_threshold"],
#     widgets={
#         "x_precedence": {"background": "#fac400"},
#         "y_precedence": {"background": "#07d900"},
#         "z_precedence": {"background": "#00c0d9"},
#     },
#     show_name=False
# )
#
# interactive_precedence_app = pn.Column(
#     "## Precedence Example",
#     "Moving the sliders of the control app should update the display of the dummy app.",
#     pn.Row(
#         pn.Column("**Control app**", control_app),
#         pn.Column("**Dummy app**", precedence_model.dummy_app)
#     )
# )
# interactive_precedence_app.show();

Launching server at http://localhost:57336
